In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'depth'
field1='Depth [Nom]'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
fields = [field1]
df_cur, df_custom=parameters(customer_id, formatted_attribute, field1)

1698


In [3]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']

In [4]:
df_custom=custom_field(df_custom,field1,formatted_attribute)
matchcust=df_custom[['external_id',curation_col]]
print(len(matchcust))
rounding(df_custom,curation_col)

16


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Depth [Nom], match, Q:depth, rounding]
Index: []

In [5]:
print(len(df_cur))

1682


# General Regex

In [6]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern=r'''(?i)((?<!LabelManager)(?<!\-)(?<!\d)\d+\.\d+[\W]{0,3}de?p?t?h?(?!ia)\b)|(\d+.?\d+\/\d+[\W]{0,3}de?p?t?h?(?!ia)\b)|((?<!LabelManager.)(?<!\-)(?<!\d)(?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\\"\-\d\d)[\W]{0,3}de?p?t?h?(?!ia)\b)|()'''           
df_cur['depth'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?x.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?x?.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)?\s?(?:[LWHD]|diam)?\s?)|(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inch|foot|fe?e?t|”)\s?(?:[LWHD]|diam)?\s?|()'''
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Depth

In [7]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!LabelManager)(?<!up.to.)(?<!\-)(?<!\d)(?<!\.)(?<!\/)(?<![\w])(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}de?p?t?h?(?!ia)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
height_new=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del height_new['new_matches']

height_new['matches']=height_new['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
height_new['matchez']=height_new['matches'].apply(lambda x: re.sub(r'''(?i)"?\s?de?p?t?h?''',' in',str(x)))


curate(height_new, 'matchez', formatted_attribute)
print(len(height_new))
matchesnew=height_new[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(height_new, 'matchez')

13


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, Depth [Nom], matches_length, matches_height, matches_width, matches_diameter, depth, blank_matches, blank_matches_long, matches, matchez, Q:depth, rounding]
Index: []

# N/A Values

In [8]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}['"].{0,2}[LWHD]?)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}['"].{0,2}[LWHD]?)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df_cur
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))

df_na=df_na[(df_na['matches_height'].astype(str)=='[]')&(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')&(df_na['blank_matches_long'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
# matchesna

243


In [9]:
df_cur[0:500]   

buckets external_id  \
0                                       Activity Tables  5637508326   
1                                       Activity Tables  5637508368   
2                                       Activity Tables  5637508408   
3                                       Activity Tables  5637508409   
4                                       Activity Tables  5637508410   
5                                       Activity Tables  5637508411   
6                                       Activity Tables  5637508412   
7                                       Activity Tables  5637508413   
8                                       Activity Tables  5637508414   
9                                       Activity Tables  5637508415   
10                                      Activity Tables  5637508416   
11                                      Activity Tables  5637508417   
12                                      Activity Tables  5637508449   
13                                      Activity Tables  5637508451   
14                                      Activity Tables  5637508452   
15                                      Activity Tables  5637508454   
16                                      Activity Tables  5637508455   
17                                      Activity Tables  5637508456   
18                                      Activity Tables  5637576681   
19                                      Activity Tables  5637576683   
20                                      Activity Tables  5637576684   
21                                      Activity Tables  5637576679   
22                                      Activity Tables  5637576685   
23                                    Bags/Cases/Covers  5637228558   
24                                    Bags/Cases/Covers  5637326571   
25                                    Bags/Cases/Covers  5637326572   
26                                    Bags/Cases/Covers  5637326573   
27                                        Refrigerators  5637352477   
28                                        Refrigerators  5637352469   
29                                        Refrigerators  5637352470   
30                                        Refrigerators  5637418034   
31                                        Refrigerators  5637418048   
32                                        Refrigerators  5637418049   
33                                        Refrigerators  5637418050   
34                                        Refrigerators  5637418051   
35                                        Refrigerators  5637418052   
36                                        Refrigerators  5637418053   
37                                        Refrigerators  5637418054   
38                                        Refrigerators  5637418055   
39                                        Refrigerators  5637418056   
40                                        Refrigerators  5637418057   
41                                        Refrigerators  5637418058   
42                                        Refrigerators  5637418059   
43                                        Refrigerators  5637418060   
44                                        Refrigerators  5637418061   
45                                        Refrigerators  5637418062   
46                                        Refrigerators  5637418063   
47                                        Refrigerators  5637418064   
48                                        Refrigerators  5637418065   
49                                        Refrigerators  5637418066   
50                                        Refrigerators  5637418067   
51                                        Refrigerators  5637418068   
52                                        Refrigerators  5637418069   
53                                        Refrigerators  5637418635   
54                                        Refrigerators  5637470015   
55                                        Refrigerators  5637470016   
5

In [8]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchesnew
matchcust
matchesna


In [9]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnew)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcust)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)